# Alternative Models: Eager Learners II

<hr>

* Support Vector Machine
    * SVC
    * NuSVC
    * LinearSVC

In [1]:
# Import needed libraries and modules
from codecarbon import EmissionsTracker
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn import svm
import optuna
from optuna.samplers import GPSampler
from sklearn.pipeline import Pipeline

# Fetch dataset from UCI Repository
from ucimlrepo import fetch_ucirepo
heart_disease = fetch_ucirepo(id=45)
df = heart_disease.data.original

In [2]:
# ---------------------------------------------------------------------------- #
#                                PRE-PROCESSING                                #
# ---------------------------------------------------------------------------- #

##### SETTINGS #####
PC_Features = True
Random_Seed = 82024
K_Folds = 10
Max_Iterations = 200
####################

# Drop missing values
df = df.dropna()
df = df.reset_index(drop=True)

# Binarize target
df.loc[df['num'] != 0, 'num'] = 1

# Define features and target vectors
X = df.iloc[:,:-1]
y = df['num']

# Separate integer from categorical features
int_features, cat_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak'],\
['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']

# Define preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('int', StandardScaler(), int_features),
        ('cat', OneHotEncoder(), cat_features)
    ])

<hr>

## Support Vector Machines:

Suport Vector Machines(SVMs) are machine learning algorithms that work by finding a hyperplane to separate data into classes

**The advantages of support vector machines are:**

* Effective in high dimensional spaces.

* Still effective in cases where number of dimensions is greater than the number of samples.

* Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.

* Versatile: different Kernel functions can be specified for the decision function as in Gaussian Process Models.

**The disadvantages of support vector machines include:**

* If the number of features is much greater than the number of samples, avoid over-fitting in choosing Kernel functions and regularization term is crucial.

* SVMs do not directly provide probability estimates.

<hr>

### SVC classification SVM implementation:

In [3]:
# ---------------------------------------------------------------------------- #
#                                     MODEL                                    #
# ---------------------------------------------------------------------------- #

# Initiate CodeCarbon to track emissions
tracker = EmissionsTracker('SVC SVM model', log_level='warning')
tracker.start()

# Create model
model = svm.SVC(random_state=Random_Seed, probability=True)

# Define pipeline depending on whether PCA is requested or not
if PC_Features:
    steps = [
        ('preprocessor', preprocessor),
        ('pca', PCA(n_components=12)),
        ('SVM-SVC-classifier', model)
    ]
else:
    steps = [
        ('preprocessor', preprocessor),
        ('SVM-SVC-classifier', model)
    ]
    
pipeline = Pipeline(steps)

# Perform cross-validation
cv_results = cross_validate(pipeline, X, y, scoring=['accuracy', 'roc_auc'], cv=K_Folds)

# Calculate and display results
acc = np.mean(cv_results['test_accuracy'])
acc_std = np.std(cv_results['test_accuracy'])
roc_auc = np.mean(cv_results['test_roc_auc'])
roc_auc_std = np.std(cv_results['test_roc_auc'])

print(f"Accuracy: {acc:.4f} ± {acc_std:.4f}")
print(f"AUC-ROC: {roc_auc:.4f} ± {roc_auc_std:.4f}")

# Stop emission tracking
_ = tracker.stop()

[codecarbon ERROR @ 14:16:28] Error: Another instance of codecarbon is already running. Turn off the other instance to be able to run this one. Exiting.


[codecarbon WARNING @ 14:16:28] Another instance of codecarbon is already running. Exiting.
[codecarbon WARNING @ 14:16:28] Another instance of codecarbon is already running. Exiting.


Accuracy: 0.8314 ± 0.0624
AUC-ROC: 0.8991 ± 0.0426


In [4]:
# Save evaluation to an external file
file = 'scores.csv'

results = pd.DataFrame({
    'SVM-SVC-model': [acc, acc_std, roc_auc, roc_auc_std]
}, index = ['Accuracy', 'Accuracy STD', 'AUC-ROC', 'AUC-ROC STD'])

# Check if file exists
if os.path.exists(file):
    temp = pd.read_csv(file, index_col=0)
    results = pd.concat([temp, results], axis=1)

# Export
results.to_csv(file)

In [5]:
# ---------------------------------------------------------------------------- #
#                                 OPTIMIZATION                                 #
# ---------------------------------------------------------------------------- #

# Initiate CodeCarbon to track emissions
tracker = EmissionsTracker('SVC SVM optimization', log_level='warning')
tracker.start()

# Objective function
def objective(trial):
      # Define parameters
      C = trial.suggest_float('C', 1e-5, 1e2, log=True)  # Faixa de valores para o parâmetro de regularização C
      kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])  # Seleção do kernel
      gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])  # Opções para o parâmetro gamma

      # Create model
      temp_model = svm.SVC(C=C, kernel=kernel, gamma=gamma)

      # Define pipeline depending on whether PCA is requested or not
      if PC_Features:
            steps = [
            ('preprocessor', preprocessor),
            ('pca', PCA(n_components=12)),
            ('SVM-SVC-classifier', temp_model)
            ]
      else:
            steps = [
            ('preprocessor', preprocessor),
            ('SVM-SVC-classifier', temp_model)
            ]
            
      pipeline = Pipeline(steps)

      # Perform cross-validation
      roc_auc = cross_val_score(pipeline, X, y, scoring='roc_auc', cv=K_Folds).mean()

      return roc_auc

# Optimizing with Optuna
study = optuna.create_study(direction='maximize', sampler=GPSampler())
study.optimize(objective, n_trials=100)

[codecarbon ERROR @ 14:16:28] Error: Another instance of codecarbon is already running. Turn off the other instance to be able to run this one. Exiting.
[codecarbon WARNING @ 14:16:28] Another instance of codecarbon is already running. Exiting.
/tmp/ipykernel_15723/2045152400.py:40: ExperimentalWarning: GPSampler is experimental (supported from v3.6.0). The interface can change in the future.
  study = optuna.create_study(direction='maximize', sampler=GPSampler())
[I 2024-10-28 14:16:29,988] A new study created in memory with name: no-name-9df31801-9e72-446c-baf5-e4201e859afb
[I 2024-10-28 14:16:30,136] Trial 0 finished with value: 0.8830700549450551 and parameters: {'C': 0.000516781902227192, 'kernel': 'poly', 'gamma': 'auto'}. Best is trial 0 with value: 0.8830700549450551.
[I 2024-10-28 14:16:30,278] Trial 1 finished with value: 0.8635989010989011 and parameters: {'C': 16.2762734344627, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 0 with value: 0.8830700549450551.
[I 2024-10-28 

In [6]:
# Show best model
trial = study.best_trial
print('AUC-ROC: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

AUC-ROC: 0.917032967032967
Best hyperparameters: {'C': 0.7407947572427896, 'kernel': 'sigmoid', 'gamma': 'scale'}


In [7]:
# Visualizing optimization
optuna.visualization.plot_optimization_history(study)

In [8]:
optuna.visualization.plot_slice(study)

In [9]:
# Create optimized model
best_params = trial.params
model = svm.SVC(random_state=Random_Seed, probability=True, **best_params)

# Define pipeline depending on whether PCA is requested or not
if PC_Features:
    steps = [
        ('preprocessor', preprocessor),
        ('pca', PCA(n_components=12)),
        ('SVM-SVC-classifier', model)
    ]
else:
    steps = [
        ('preprocessor', preprocessor),
        ('SVM-SVC-classifier', model)
    ]
    
pipeline = Pipeline(steps)

# Perform cross-validation
cv_results = cross_validate(pipeline, X, y, scoring=['accuracy', 'roc_auc'], cv=K_Folds)

# Calculate and display results
acc = np.mean(cv_results['test_accuracy'])
acc_std = np.std(cv_results['test_accuracy'])
roc_auc = np.mean(cv_results['test_roc_auc'])
roc_auc_std = np.std(cv_results['test_roc_auc'])

print(f"Accuracy: {acc:.4f} ± {acc_std:.4f}")
print(f"AUC-ROC: {roc_auc:.4f} ± {roc_auc_std:.4f}")

# Stop emission tracking
_ = tracker.stop()

[codecarbon WARNING @ 14:17:07] Another instance of codecarbon is already running. Exiting.


Accuracy: 0.8448 ± 0.0512
AUC-ROC: 0.9170 ± 0.0348


In [10]:
# Save evaluation to an external file
file = 'scores.csv'

results = pd.DataFrame({
    'SVM-SVC-optimization': [acc, acc_std, roc_auc, roc_auc_std]
}, index = ['Accuracy', 'Accuracy STD', 'AUC-ROC', 'AUC-ROC STD'])

# Check if file exists
if os.path.exists(file):
    temp = pd.read_csv(file, index_col=0)
    results = pd.concat([temp, results], axis=1)

# Export
results.to_csv(file)

<hr>

### NuSVC classification SVM implementation:

In [11]:
# ---------------------------------------------------------------------------- #
#                                     MODEL                                    #
# ---------------------------------------------------------------------------- #

# Initiate CodeCarbon to track emissions
tracker = EmissionsTracker('NuSVC SVM model', log_level='warning')
tracker.start()

# Create model
model = svm.NuSVC(random_state=Random_Seed, probability=True)

# Define pipeline depending on whether PCA is requested or not
if PC_Features:
    steps = [
        ('preprocessor', preprocessor),
        ('pca', PCA(n_components=12)),
        ('SVM-NuSVC-classifier', model)
    ]
else:
    steps = [
        ('preprocessor', preprocessor),
        ('SVM-NuSVC-classifier', model)
    ]
    
pipeline = Pipeline(steps)

# Perform cross-validation
cv_results = cross_validate(pipeline, X, y, scoring=['accuracy', 'roc_auc'], cv=K_Folds)

# Calculate and display results
acc = np.mean(cv_results['test_accuracy'])
acc_std = np.std(cv_results['test_accuracy'])
roc_auc = np.mean(cv_results['test_roc_auc'])
roc_auc_std = np.std(cv_results['test_roc_auc'])

print(f"Accuracy: {acc:.4f} ± {acc_std:.4f}")
print(f"AUC-ROC: {roc_auc:.4f} ± {roc_auc_std:.4f}")

# Stop emission tracking
_ = tracker.stop()

[codecarbon ERROR @ 14:17:07] Error: Another instance of codecarbon is already running. Turn off the other instance to be able to run this one. Exiting.
[codecarbon WARNING @ 14:17:07] Another instance of codecarbon is already running. Exiting.
[codecarbon WARNING @ 14:17:07] Another instance of codecarbon is already running. Exiting.


Accuracy: 0.8282 ± 0.0611
AUC-ROC: 0.9009 ± 0.0440


In [12]:
# Save evaluation to an external file
file = 'scores.csv'

results = pd.DataFrame({
    'SVM-NuSVC-model': [acc, acc_std, roc_auc, roc_auc_std]
}, index = ['Accuracy', 'Accuracy STD', 'AUC-ROC', 'AUC-ROC STD'])

# Check if file exists
if os.path.exists(file):
    temp = pd.read_csv(file, index_col=0)
    results = pd.concat([temp, results], axis=1)

# Export
results.to_csv(file)

In [13]:
# ---------------------------------------------------------------------------- #
#                                 OPTIMIZATION                                 #
# ---------------------------------------------------------------------------- #

# Initiate CodeCarbon to track emissions
tracker = EmissionsTracker('NuSVC SVM optimization', log_level='warning')
tracker.start()

# Objective function
def objective(trial):
  # Define parameters
  nu = trial.suggest_float('nu', 0.1, 0.9, log=True)  # nu é um parâmetro entre 0 e 1
  kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])  # Tipo de kernel
  gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])  # Gamma para os kernels não lineares

  # Create model
  temp_model = svm.NuSVC(nu=nu, kernel=kernel, gamma=gamma)
  
  # Define pipeline depending on whether PCA is requested or not
  if PC_Features:
        steps = [
        ('preprocessor', preprocessor),
        ('pca', PCA(n_components=12)),
        ('SVM-NuSVC-classifier', temp_model)
        ]
  else:
        steps = [
        ('preprocessor', preprocessor),
        ('SVM-NuSVC-classifier', temp_model)
        ]
        
  pipeline = Pipeline(steps)

  # Perform cross-validation
  roc_auc = cross_val_score(pipeline, X, y, scoring='roc_auc', cv=K_Folds).mean()
  
  return roc_auc

# Optimizing with Optuna
study = optuna.create_study(direction='maximize', sampler=GPSampler())
study.optimize(objective, n_trials=100)

[codecarbon ERROR @ 14:17:07] Error: Another instance of codecarbon is already running. Turn off the other instance to be able to run this one. Exiting.
[codecarbon WARNING @ 14:17:07] Another instance of codecarbon is already running. Exiting.
/tmp/ipykernel_15723/2282820642.py:40: ExperimentalWarning:

GPSampler is experimental (supported from v3.6.0). The interface can change in the future.

[I 2024-10-28 14:17:07,982] A new study created in memory with name: no-name-2328c26f-9e51-43f3-9a63-c39f1d7eabb2
[I 2024-10-28 14:17:08,133] Trial 0 finished with value: 0.8947802197802197 and parameters: {'nu': 0.6731675018697041, 'kernel': 'rbf', 'gamma': 'scale'}. Best is trial 0 with value: 0.8947802197802197.
[I 2024-10-28 14:17:08,271] Trial 1 finished with value: 0.8623626373626374 and parameters: {'nu': 0.1676535980560012, 'kernel': 'rbf', 'gamma': 'scale'}. Best is trial 0 with value: 0.8947802197802197.
[I 2024-10-28 14:17:08,399] Trial 2 finished with value: 0.8968063186813188 and pa

In [14]:
# Show best model
trial = study.best_trial
print('AUC-ROC: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

AUC-ROC: 0.9141483516483516
Best hyperparameters: {'nu': 0.5875499385362177, 'kernel': 'linear', 'gamma': 'auto'}


In [15]:
# Visualizing optimization
optuna.visualization.plot_optimization_history(study)

In [16]:
optuna.visualization.plot_slice(study)

In [17]:
# Create optimized model
best_params = trial.params
model = svm.NuSVC(random_state=Random_Seed, probability=True, **best_params)

# Define pipeline depending on whether PCA is requested or not
if PC_Features:
    steps = [
        ('preprocessor', preprocessor),
        ('pca', PCA(n_components=12)),
        ('SVM-NuSVC-classifier', model)
    ]
else:
    steps = [
        ('preprocessor', preprocessor),
        ('SVM-NuSVC-classifier', model)
    ]
    
pipeline = Pipeline(steps)

# Perform cross-validation
cv_results = cross_validate(pipeline, X, y, scoring=['accuracy', 'roc_auc'], cv=K_Folds)

# Calculate and display results
acc = np.mean(cv_results['test_accuracy'])
acc_std = np.std(cv_results['test_accuracy'])
roc_auc = np.mean(cv_results['test_roc_auc'])
roc_auc_std = np.std(cv_results['test_roc_auc'])

print(f"Accuracy: {acc:.4f} ± {acc_std:.4f}")
print(f"AUC-ROC: {roc_auc:.4f} ± {roc_auc_std:.4f}")

# Stop emission tracking
_ = tracker.stop()

[codecarbon WARNING @ 14:17:38] Another instance of codecarbon is already running. Exiting.


Accuracy: 0.8414 ± 0.0592
AUC-ROC: 0.9141 ± 0.0350


In [18]:
# Save evaluation to an external file
file = 'scores.csv'

results = pd.DataFrame({
    'SVM-NuSVC-optimization': [acc, acc_std, roc_auc, roc_auc_std]
}, index = ['Accuracy', 'Accuracy STD', 'AUC-ROC', 'AUC-ROC STD'])

# Check if file exists
if os.path.exists(file):
    temp = pd.read_csv(file, index_col=0)
    results = pd.concat([temp, results], axis=1)

# Export
results.to_csv(file)

<hr>

### LinearSVC classification SVM implementation:

In [19]:
# ---------------------------------------------------------------------------- #
#                                     MODEL                                    #
# ---------------------------------------------------------------------------- #

# Initiate CodeCarbon to track emissions
tracker = EmissionsTracker('LinearSVC SVM model', log_level='warning')
tracker.start()

# Create model
model = svm.LinearSVC(random_state=Random_Seed, dual=False)

# Define pipeline depending on whether PCA is requested or not
if PC_Features:
    steps = [
        ('preprocessor', preprocessor),
        ('pca', PCA(n_components=12)),
        ('SVM-LinearSVC-classifier', model)
    ]
else:
    steps = [
        ('preprocessor', preprocessor),
        ('SVM-LinearSVC-classifier', model)
    ]
    
pipeline = Pipeline(steps)

# Perform cross-validation
cv_results = cross_validate(pipeline, X, y, scoring=['accuracy', 'roc_auc'], cv=K_Folds)

# Calculate and display results
acc = np.mean(cv_results['test_accuracy'])
acc_std = np.std(cv_results['test_accuracy'])
roc_auc = np.mean(cv_results['test_roc_auc'])
roc_auc_std = np.std(cv_results['test_roc_auc'])

print(f"Accuracy: {acc:.4f} ± {acc_std:.4f}")
print(f"AUC-ROC: {roc_auc:.4f} ± {roc_auc_std:.4f}")

# Stop emission tracking
_ = tracker.stop()

[codecarbon ERROR @ 14:17:38] Error: Another instance of codecarbon is already running. Turn off the other instance to be able to run this one. Exiting.
[codecarbon WARNING @ 14:17:38] Another instance of codecarbon is already running. Exiting.


[codecarbon WARNING @ 14:17:39] Another instance of codecarbon is already running. Exiting.


Accuracy: 0.8448 ± 0.0512
AUC-ROC: 0.9124 ± 0.0336


In [20]:
# Save evaluation to an external file
file = 'scores.csv'

results = pd.DataFrame({
    'SVM-LinearSVC-model': [acc, acc_std, roc_auc, roc_auc_std]
}, index = ['Accuracy', 'Accuracy STD', 'AUC-ROC', 'AUC-ROC STD'])

# Check if file exists
if os.path.exists(file):
    temp = pd.read_csv(file, index_col=0)
    results = pd.concat([temp, results], axis=1)

# Export
results.to_csv(file)

In [21]:
# ---------------------------------------------------------------------------- #
#                                 OPTIMIZATION                                 #
# ---------------------------------------------------------------------------- #

# Initiate CodeCarbon to track emissions
tracker = EmissionsTracker('LinearSVC SVM optimization', log_level='warning')
tracker.start()

# Objective function
def objective(trial):
    # Define parameters
    C = trial.suggest_float('C', 1e-5, 1e2, log=True)

    # Create model
    temp_model = svm.LinearSVC(C=C, dual=False)

    # Define pipeline depending on whether PCA is requested or not
    if PC_Features:
        steps = [
        ('preprocessor', preprocessor),
        ('pca', PCA(n_components=12)),
        ('SVM-LinearSVC-classifier', temp_model)
        ]
    else:
        steps = [
        ('preprocessor', preprocessor),
        ('SVM-LinearSVC-classifier', temp_model)
        ]
        
    pipeline = Pipeline(steps)

    # Perform cross-validation
    roc_auc = cross_val_score(pipeline, X, y, scoring='roc_auc', cv=K_Folds).mean()

    return roc_auc

# Optimizing with Optuna
study = optuna.create_study(direction='maximize', sampler=GPSampler())
study.optimize(objective, n_trials=100)

[codecarbon ERROR @ 14:17:39] Error: Another instance of codecarbon is already running. Turn off the other instance to be able to run this one. Exiting.
[codecarbon WARNING @ 14:17:39] Another instance of codecarbon is already running. Exiting.
/tmp/ipykernel_15723/3847851053.py:38: ExperimentalWarning:

GPSampler is experimental (supported from v3.6.0). The interface can change in the future.

[I 2024-10-28 14:17:39,072] A new study created in memory with name: no-name-dd4aedac-9ead-4949-affd-acda2238e0d0
[I 2024-10-28 14:17:39,207] Trial 0 finished with value: 0.9123969780219781 and parameters: {'C': 0.8593281829330846}. Best is trial 0 with value: 0.9123969780219781.
[I 2024-10-28 14:17:39,314] Trial 1 finished with value: 0.9128434065934066 and parameters: {'C': 60.76571384941635}. Best is trial 1 with value: 0.9128434065934066.
[I 2024-10-28 14:17:39,423] Trial 2 finished with value: 0.9123969780219781 and parameters: {'C': 2.021251523488277}. Best is trial 1 with value: 0.9128434

In [22]:
# Show best model
trial = study.best_trial
print('AUC-ROC: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

AUC-ROC: 0.9152472527472527
Best hyperparameters: {'C': 0.07760733796507535}


In [23]:
# Visualizing optimization
optuna.visualization.plot_optimization_history(study)

In [24]:
optuna.visualization.plot_slice(study)

In [25]:
# Create optimized model
best_params = trial.params
model = svm.LinearSVC(random_state=Random_Seed, dual=False, **best_params)

# Define pipeline depending on whether PCA is requested or not
if PC_Features:
    steps = [
        ('preprocessor', preprocessor),
        ('pca', PCA(n_components=12)),
        ('SVM-LinearSVC-classifier', model)
    ]
else:
    steps = [
        ('preprocessor', preprocessor),
        ('SVM-LinearSVC-classifier', model)
    ]
    
pipeline = Pipeline(steps)

# Perform cross-validation
cv_results = cross_validate(pipeline, X, y, scoring=['accuracy', 'roc_auc'], cv=K_Folds)

# Calculate and display results
acc = np.mean(cv_results['test_accuracy'])
acc_std = np.std(cv_results['test_accuracy'])
roc_auc = np.mean(cv_results['test_roc_auc'])
roc_auc_std = np.std(cv_results['test_roc_auc'])

print(f"Accuracy: {acc:.4f} ± {acc_std:.4f}")
print(f"AUC-ROC: {roc_auc:.4f} ± {roc_auc_std:.4f}")

# Stop emission tracking
_ = tracker.stop()

[codecarbon WARNING @ 14:18:23] Another instance of codecarbon is already running. Exiting.


Accuracy: 0.8515 ± 0.0576
AUC-ROC: 0.9152 ± 0.0345


In [26]:
# Save evaluation to an external file
file = 'scores.csv'

results = pd.DataFrame({
    'SVM-LinearSVC-optimization': [acc, acc_std, roc_auc, roc_auc_std]
}, index = ['Accuracy', 'Accuracy STD', 'AUC-ROC', 'AUC-ROC STD'])

# Check if file exists
if os.path.exists(file):
    temp = pd.read_csv(file, index_col=0)
    results = pd.concat([temp, results], axis=1)

# Export
results.to_csv(file)

### References:
* https://scikit-learn.org/stable/modules/svm.html